In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score, f1_score,classification_report
from random import randint, shuffle
import os
import seaborn as sns




# cluster0と1を結合して、test_dataの結果と比較する

In [11]:
train_data = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/train_data/train_data.csv')
test_data = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/test_data/test_data.csv')

In [2]:
#clusterごとに分けたデータの予測結果追加を読み込み
train_data_cluster0_pred = pd.read_csv('Desktop/インテグ課題/NASA/train_data/train_data_cluster0_pred.csv')
test_data_cluster0_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_cluster0_pred.csv')

In [3]:
#clusterごとに分けたデータの予測結果追加を読み込み
train_data_cluster1_pred = pd.read_csv('Desktop/インテグ課題/NASA/train_data/train_data_cluster1_pred.csv')
test_data_cluster1_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_cluster1_pred.csv')

In [4]:
#2つのdataを結合する
train_data_all_pred = train_data_cluster0_pred.append(train_data_cluster1_pred)
test_data_all_pred = test_data_cluster0_pred.append(test_data_cluster1_pred)

In [5]:
# 2つのataを最初の形に並べ替える
train_data_all_pred = train_data_all_pred.sort_values(['engine_no', 'time_in_cycles'])
test_data_all_pred = test_data_all_pred.sort_values(['engine_no', 'time_in_cycles'])


In [7]:
#2つのtdataを結合したデータの予測結果追加をを出力
train_data_all_pred.to_csv('Desktop/インテグ課題/NASA/train_data/train_data_all_pred.csv', index=False)
test_data_all_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_all_pred.csv', index=False)


In [8]:
train_data_all_pred.head()

,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,RUL,pred_rf_fe_im,pred_rf_gscv_fe_im_tu_all
0,0,1,25.0074,0.6200,60.0,462.54,536.84,1256.52,1043.97,7.05,...,10.8850,0.02,306,1915,84.93,14.35,8.4712,339,231.5,267.035
1,0,2,35.0072,0.8413,100.0,449.44,555.44,1364.42,1128.75,5.48,...,9.2527,0.02,334,2223,100.00,14.88,8.9928,338,338.0,291.060
2,0,3,25.0053,0.6215,60.0,462.54,536.42,1265.94,1047.23,7.05,...,10.9054,0.02,307,1915,84.93,14.21,8.5107,337,291.8,260.865
3,0,4,42.0045,0.8407,100.0,445.00,549.41,1355.52,1115.81,3.91,...,9.3434,0.02,331,2212,100.00,10.63,6.4578,336,301.1,286.525
4,0,5,35.0046,0.8400,100.0,449.44,555.21,1361.04,1123.63,5.48,...,9.3287,0.02,334,2223,100.00,14.95,9.0279,335,264.4,270.795


# 結合したdataのレコード数が元のdataのレコード数と同じか確認する

In [12]:
print(train_data.shape)
print(train_data_all_pred.shape)

(160359, 33)
(160359, 29)


In [15]:
print(test_data.shape)
print(test_data_all_pred.shape)

(104897, 32)
(104897, 29)


In [16]:
#結果表の作成 
# train_data_all_pred = train_data_all_pred.sort_values(['engine_no', 'time_in_cycles'])
# test_data_all_pred = test_data_all_pred.sort_values(['engine_no', 'time_in_cycles'])

#test_dataの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_rf_gscv_all'のみ
result_test_data_all_pred = test_data_all_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_rf_gscv_fe_im_tu_all']]



In [17]:
result_test_data_all_pred.head()

,engine_no,pred_rf_gscv_fe_im_tu_all
0,0,28.605
1,1,29.490
2,2,169.705
3,3,120.165
4,4,9.340


In [18]:
# pred_rf_gscv_fe_im_tu_all<100の時、つまり予測した残りの寿命サイクルが100より小さければ1 それ以外で0とする
result_test_data_all_pred['result'] = result_test_data_all_pred['pred_rf_gscv_fe_im_tu_all']\
.map(lambda x: 0 if x > 100 else 1)

In [19]:
result_test_data_all_pred.head()

,engine_no,pred_rf_gscv_fe_im_tu_all,result
0,0,28.605,1
1,1,29.490,1
2,2,169.705,0
3,3,120.165,0
4,4,9.340,1


In [20]:
#csvで結果の保存
result_test_data_all_pred[['engine_no', 'result']].to_csv('Desktop/インテグ課題/NASA/test_data/result_test_data_all_pred.csv', index=False)

# test_dataのRULを読み込んで処理

In [21]:
#testデータのRULを読み込む
test_data_RUL = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/RUL_test.csv')

In [22]:
test_data_RUL.head()

,Unnamed: 0,RUL,engine_no
0,0,22,0
1,1,12,1
2,2,108,2
3,3,88,3
4,4,10,4


In [23]:
#列の順番を入れ替えと削除
test_data_RUL.drop('Unnamed: 0', axis=1)
test_data_RUL = test_data_RUL.loc[:,['engine_no','RUL']]
test_data_RUL.head()

,engine_no,RUL
0,0,22
1,1,12
2,2,108
3,3,88
4,4,10


In [24]:
# RUL<100の時、つまり予測した残りの寿命サイクルが100より小さければ1 それ以外で0とする
test_data_RUL['result'] = test_data_RUL['RUL'].map(lambda x: 0 if x > 100 else 1)

In [25]:
test_data_RUL.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


In [26]:
print((test_data_RUL['result'] == 1).sum())
print((test_data_RUL['result'] == 0).sum())

450
257


# result_test_data_all_predの特徴量選択とパラメータ調整後のデータを使う

In [27]:
print((result_test_data_all_pred['result'] == 1).sum())
print((result_test_data_all_pred['result'] == 0).sum())

350
357


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [28]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],result_test_data_all_pred['result'],labels=[1, 0] )

In [29]:
cm

array([[332, 118],
       [ 18, 239]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [30]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],result_test_data_all_pred['result'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.95      0.74      0.83       450
           0       0.67      0.93      0.78       257

    accuracy                           0.81       707
   macro avg       0.81      0.83      0.80       707
weighted avg       0.85      0.81      0.81       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方